<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/%E5%9B%BD%E4%BC%9A%E8%AD%B0%E4%BA%8B%E9%8C%B2%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install requests

## 議事録の著作権について

- 電子計算機による情報解析については、著作権法の定めに従い、著作権者の許諾を得ることなく利用可能
- ソース https://kokkai.ndl.go.jp/help.html#:~:text=%E5%80%8B%E3%80%85%E3%81%AE%E7%99%BA%E8%A8%80%E3%81%AE%E3%81%86%E3%81%A1,%E8%A8%B1%E8%AB%BE%E3%81%8C%E5%BF%85%E8%A6%81%E3%81%A8%E3%81%AA%E3%82%8A%E3%81%BE%E3%81%99%E3%80%82

### 参考URL
https://qiita.com/8_hisakichi_8/items/d6894803d2ebabf3d33b


## 国会図書館APIから議事録を抽出

## JSON形式版

In [71]:
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

# 処理結果を自動的にダウンロードするように変更

# 保存用ディレクトリの指定
submit_dir = "/content/drive/MyDrive/gijiroku/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
import json
import time
from urllib.request import urlopen, Request
from urllib.parse import quote
from urllib.error import URLError, HTTPError
import xml.etree.ElementTree as ET
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
import os

def create_query(session, meeting_name, issue):
    params = {
        'nameOfMeeting': meeting_name,
        'maximumRecords': 1,
        "sessionFrom": session,
        "sessionTo": session,
        "issueFrom": issue,
        "issueTo": issue,
    }
    return '&'.join(['{}={}'.format(key, value) for key, value in params.items()])

def parse_xml(res_xml):
    root = ET.fromstring(res_xml)
    speech_list = []
    try:
        for record in root.findall('./records/record/recordData/meetingRecord'):
            session = record.find('session').text
            nameOfMeeting = record.find('nameOfMeeting').text
            issue = record.find('issue').text
            date = record.find('date').text
            print(nameOfMeeting, issue, date)
            i = 1
            chairman = None
            for speechRecord in record.findall('speechRecord'):
                speaker_group = speechRecord.find("speakerGroup").text
                speaker = speechRecord.find('speaker').text
                contents = speechRecord.find('speech').text
                if not any([speaker is None, speaker_group is None]):
                    sep = "　"
                    contents = sep.join(contents.split(sep)[1:])
                    speech = {}
                    speech["発言No."] = i
                    speech["発言者"] = speaker
                    speech["発言内容"] = contents
                    speech["国会会期"] = session
                    speech["会議名"] = nameOfMeeting
                    speech["会議号"] = issue
                    speech["会議日付"] = date
                    speech_list.append(speech)
                    i += 1
    except ET.ParseError as e:
        print('ParseError: {}'.format(e.code))
    return speech_list

def process_speech_list(speech_list):
    # 発言者ごとに発言内容をまとめて表示
    for speech in speech_list:
        print(f"発言No.: {speech['発言No.']}")
        print(f"発言者: {speech['発言者']}")
        print(f"発言内容: {speech['発言内容']}")
        print("-" * 30)

def main():
    # 保存先のディレクトリ
    submit_dir = "/content/drive/MyDrive/gijiroku/"

    # フォルダが存在しない場合には作成
    if not os.path.exists(submit_dir):
      os.makedirs(submit_dir)

    # meeting_name = '本会議'
    meeting_name = '予算委員会'
    session = 208
    issue = 1
    # 取得した国会発言をmain外で色々処理をするためグローバル変数で宣言
    global minutes
    minutes = []  # minutesをリストとして初期化

    while True:
        query = create_query(session, meeting_name, issue)
        # ここでAPIリクエストを送信して議事録を取得する処理を実装する
        # クエリはパーセントエンコードしておく
        request_url = 'http://kokkai.ndl.go.jp/api/1.0/meeting?' + quote(create_query(session=session,
                                                                                   meeting_name=meeting_name,
                                                                                   issue=issue))
        req = Request(request_url)
        try:
            # リクエストクエリのURLからオブジェクト（今回はXML）を取得しutf8にデコード
            with urlopen(req) as res:
                res_xml = res.read().decode('utf8')

        except HTTPError as e:
            print('HTTPError: {}'.format(e.reason))
        except URLError as e:
            print('URLError: {}'.format(e.reason))

        # try正常終了時の処理記述
        else:
            # 取得したxmlを国会発言として扱いやすい形（辞書のリスト）に変換
            minutes_issue = parse_xml(res_xml)

            # レスポンスが無い場合にループを抜け出す
            if not minutes_issue:
                break
            else:
              # 取得したminutesをリストに追加
              minutes.extend(minutes_issue)

              # ここでminutesを適切に処理する
              process_speech_list(minutes_issue)

              issue += 1  # 次の議事録番号を更新
              time.sleep(3)  # インターバルを3秒に設定

    # ループが終了したら、全てのminutesをJSONファイルに書き出す
    file_name = f"{session}_{meeting_name}.json"
    with open(os.path.join(submit_dir, file_name), "w") as json_file:
        json.dump(minutes, json_file, ensure_ascii=False, indent=4)

if __name__ == '__main__':
    main()



ストリーミング出力は最後の 5000 行に切り捨てられました。
------------------------------
発言No.: 130
発言者: 山本順三
発言内容: 以上で蓮舫さんの質疑は終了いたしました。（拍手）
　　　　─────────────
------------------------------
発言No.: 131
発言者: 山本順三
発言内容: 次に、小西洋之君の質疑を行います。小西洋之君。
------------------------------
発言No.: 132
発言者: 小西洋之
発言内容: 立憲民主・社民の小西洋之でございます。
　私はまず、日本社会の皆さんが今直面をしている異次元の物価高騰ともいうべき問題について質問をさせていただきます。
　この物価高騰の原因でございますが、この円安、円安が進んでいるこの背景には日米の金利差拡大があります。
　日銀の黒田総裁にお伺いをします。
　今のこの物価の上昇、物価の高騰と、日銀が四月二十八日、進めると決定をした異次元の金融緩和、これは関係がある、この異次元の金融緩和のために物価の上昇、高騰が起きている、そういう理解でよろしいでしょうか。
------------------------------
発言No.: 133
発言者: 小西洋之
発言内容: まあ黒田総裁も答弁拒否をするんですか。じゃ、総裁、よろしいですか。
　今輸入物価が上昇している、昨年に比較して四月の値で四五％、うち一五％は円安の影響によるものである、これは黒田日銀総裁、また岸田総理も答弁していますが、そういう理解でよろしいでしょうか。
------------------------------
発言No.: 134
発言者: 小西洋之
発言内容: 今、黒田総裁は、今、この輸入物価の上昇のこの三分の一は円安が原因であると、これは日銀が出している数字なんですが、はっきりおっしゃいました。
　では、黒田総裁に伺いますが、今の円安と日銀が四月二十八日以降進めると決定した異次元の金融緩和、これは関係がありますか。
------------------------------
発言No.: 135
発言者: 小西洋之
発言内容: 岸田総理、三分の一、四分の一、ちょっと整理をさせていただきますが、岸田総理、衆議院の五月二

### 以下、ボツコード

In [ ]:
from urllib.request import Request, urlopen
from urllib.parse import quote
from urllib.error import URLError, HTTPError
import xml.etree.ElementTree as ET
import time
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

def create_query(session, meeting_name, issue):
    params = {
        'nameOfMeeting': meeting_name,
        'maximumRecords': 1,
        "sessionFrom": session,
        "sessionTo": session,
        "issueFrom": issue,
        "issueTo": issue,
    }
    return '&'.join(['{}={}'.format(key, value) for key, value in params.items()])

def parse_xml(res_xml):
    root = ET.fromstring(res_xml)
    speech_list = []
    try:
        for record in root.findall('./records/record/recordData/meetingRecord'):
            session = record.find('session').text
            nameOfMeeting = record.find('nameOfMeeting').text
            issue = record.find('issue').text
            date = record.find('date').text
            print(nameOfMeeting, issue, date)
            i = 1
            chairman = None
            for speechRecord in record.findall('speechRecord'):
                speaker_group = speechRecord.find("speakerGroup").text
                speaker = speechRecord.find('speaker').text
                contents = speechRecord.find('speech').text
                if not any([speaker is None, speaker_group is None]):
                    sep = "　"
                    contents = sep.join(contents.split(sep)[1:])
                    speech = {}
                    speech["発言No."] = i
                    speech["発言者"] = speaker
                    speech["発言内容"] = contents
                    speech["国会会期"] = session
                    speech["会議名"] = nameOfMeeting
                    speech["会議号"] = issue
                    speech["会議日付"] = date
                    speech_list.append(speech)
                    i += 1
    except ET.ParseError as e:
        print('ParseError: {}'.format(e.code))
    return speech_list

def process_speech_list(speech_list):
    # 発言者ごとに発言内容をまとめて表示
    for speech in speech_list:
        print(f"発言No.: {speech['発言No.']}")
        print(f"発言者: {speech['発言者']}")
        print(f"発言内容: {speech['発言内容']}")
        print("-" * 30)

def main():

    #meeting_name = '本会議'
    meeting_name =  '予算委員会'
    session = 208
    issue = 1
    # 取得した国会発言をmain外で色々処理をするためグローバル変数で宣言
    global minutes

    while True:
        query = create_query(session, meeting_name, issue)
        # ここでAPIリクエストを送信して議事録を取得する処理を実装する
        # クエリはパーセントエンコードしておく
        request_url = 'http://kokkai.ndl.go.jp/api/1.0/meeting?' + quote(create_query(session= session,
                                                                                      meeting_name= meeting_name,
                                                                                      issue= issue))
        req = Request(request_url)
        try:
          # リクエストクエリのURLからオブジェクト（今回はXML）を取得しutf8にデコード
          with urlopen(req) as res:
            res_xml = res.read().decode('utf8')

        except HTTPError as e:
            print('HTTPError: {}'.format(e.reason))
        except URLError as e:
            print('URLError: {}'.format(e.reason))

        # try正常終了時の処理記述
        else:
            # 取得したxmlを国会発言として扱いやすい形（クラスオブジェクトのリスト）に変換
            minutes = parse_xml(res_xml,)

            # ここでspeech_listを適切に処理する
            process_speech_list(minutes)
            issue += 1  # 次の議事録番号を更新
            time.sleep(3)  # インターバルを3秒に設定

            # レスポンスが無い場合にループを抜け出す
            if not minutes :
              break

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
if __name__ == '__main__':
    main()



In [40]:
from urllib.request import Request, urlopen
from urllib.parse import quote
from urllib.error import URLError, HTTPError
import xml.etree.ElementTree as ET

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def create_query(session, meeting_name, issue):
    """
    リクエストクエリの作成（必要情報を受け取り、URLに変換して返す）
    """
    # ）10回分の発言を取得する
    params = {
        'nameOfMeeting': meeting_name,
        'maximumRecords': 10, #10回分
        "sessionFrom": session,  # 国会の第〇期を指す。一期分の国会を対象とするためFromとToは同じに
        "sessionTo": session,
        "issueFrom": issue,  # 各会議の第〇回を指す。
        "issueTo": issue,
        }

    return '&'.join(['{}={}'.format(key, value) for key, value in params.items()])

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
def parse_xml(res_xml):
    root = ET.fromstring(res_xml)

    # 最終的に戻り値として返す発言情報をリスト型で宣言
    speech_list = []

    try:
        for record in root.findall('./records/record/recordData/meetingRecord'):
            # 会議録情報の取得
            session = record.find('session').text
            nameOfMeeting = record.find('nameOfMeeting').text
            issue = record.find('issue').text
            date = record.find('date').text
            print(nameOfMeeting, issue, date)

            i = 1
            chairman = None

            for speechRecord in record.findall('speechRecord'):
             #for i, speechRecord in enumerate(record.findall("speechRecord")):
                # 発言者と発言の取得
                speaker_group = speechRecord.find("speakerGroup").text
                speaker = speechRecord.find('speaker').text
                contents = speechRecord.find('speech').text

                # if speaker is not None and speaker_group is not None:
                # 先頭のspeechRecord（speaker=None）は出席者一覧などの会議録情報なのでスキップ
                if not any([speaker is None ,speaker_group is None]):

#                    # 議長情報がなければ最初の発言者を議長と判断
#                    if chairman is None:
#                      chairman = speaker
#                    # 議長の発言もスキップ
#                    if speaker != chairman:

                    # 最初の区切り文字以降を発言として再格納（最初の区切り文字までは基本発言者名のため）
                    sep = "　"  # 区切り文字
                    contents = sep.join(contents.split(sep)[1:])

                    # 発言情報の辞書作成
                    speech = {}
                    speech["発言No."] = i
                    speech["発言者"] = speaker
                    speech["発言者所属"] = speaker_group
                    speech["発言内容"] = contents
                    speech["国会会期"] = session
                    speech["会議名"] = nameOfMeeting
                    speech["会議号"] = issue
                    speech["会議日付"] = date

                    # 発言情報をリストに集積
                    speech_list.append(speech)

                    i += 1

    # エラー処理
    except ET.ParseError as e:
        print('ParseError: {}'.format(e.code))

    return speech_list

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
def main():

    #meeting_name = '本会議'
    meeting_name =  '予算委員会'
    session = 208
    issue = 1

    # クエリはパーセントエンコードしておく
    request_url = 'http://kokkai.ndl.go.jp/api/1.0/meeting?' + quote(create_query(session= session,
                                                                                   meeting_name= meeting_name,
                                                                                   issue= issue))
    req = Request(request_url)

    # 取得した国会発言をmain外で色々処理をするためグローバル変数で宣言
    global minutes

    try:
        # リクエストクエリのURLからオブジェクト（今回はXML）を取得しutf8にデコード
        with urlopen(req) as res:
            res_xml = res.read().decode('utf8')

    except HTTPError as e:
        print('HTTPError: {}'.format(e.reason))
    except URLError as e:
        print('URLError: {}'.format(e.reason))

    # try正常終了時の処理記述
    else:
        # 取得したxmlを国会発言として扱いやすい形（クラスオブジェクトのリスト）に変換
        minutes = parse_xml(res_xml,)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
if __name__ == '__main__':
    main()


予算委員会 第1号 2022-01-21
予算委員会 第1号 2022-01-21
予算委員会公聴会 第1号 2022-03-08
予算委員会公聴会 第1号 2022-02-15
予算委員会第一分科会 第1号 2022-02-16
予算委員会第二分科会 第1号 2022-02-16
予算委員会第三分科会 第1号 2022-02-16
予算委員会第四分科会 第1号 2022-02-16
予算委員会第五分科会 第1号 2022-02-16
予算委員会第六分科会 第1号 2022-02-16


In [ ]:
for i in minutes : print(i)